<a href="https://colab.research.google.com/github/abdulrahmanalaa123/Graduation_Project/blob/Honda/grad_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

imports


In [1]:
!pip3 install pickle5
import pickle5 as pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from re import L
import json
from operator import itemgetter
from google.colab import files
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
#from keras.layers import Dropout
#%pip install pandas==1.4.1       unpickle problem





In [ ]:
uploaded = files.upload()


Saving hr(tuples).pkl to hr(tuples).pkl


In [3]:
with open("/content/hr(tuples).pkl", "rb") as f: 
    data = pickle.load(f) 

In [9]:
#data = pd.read_pickle("/content/hr(tuples).pkl")
%pip install pandas==1.4.1

'1.2.5'

In [4]:
print(data)

      Participant           Stage       0:0.0  0:0.099791  0:0.199582  \
0              22        STIMULUS   (87, 602)   (87, 602)   (87, 602)   
1              22  QUESTIONNAIRES   (85, 921)   (85, 921)   (85, 921)   
2              22         WASHOUT   (80, 749)   (80, 792)   (80, 792)   
3              22        STIMULUS   (81, 719)   (81, 719)   (81, 719)   
4              22  QUESTIONNAIRES   (83, 702)   (82, 702)   (83, 702)   
...           ...             ...         ...         ...         ...   
1339           64        STIMULUS   (78, 516)   (78, 516)   (78, 516)   
1340           64  QUESTIONNAIRES   (77, 799)   (77, 799)   (77, 816)   
1341           64         WASHOUT    (0, 987)    (0, 987)    (0, 987)   
1342           64        STIMULUS   (69, 829)   (68, 829)   (69, 829)   
1343           64  QUESTIONNAIRES  (66, 1245)  (65, 1245)  (65, 1245)   

     0:0.299373  0:0.399164 0:0.49895500000000004  0:0.598746  \
0     (87, 602)   (87, 602)             (87, 602)   (87, 6

In [5]:
def getter(x):
    if isinstance(x,tuple):
        return itemgetter(0)(x)
    else:
        return x

#data = pd.read_pickle("hr(tuples).pkl")
#mask = data["Participant"] ==  22
mask2 = data["Stage"] == "STIMULUS"
mask3 = data["Target"]== "BASELINE"
#print(data.loc[ mask2,[*data.columns[2:500],*data.columns[-15:]]].applymap(lambda x:getter(x)))
#print(data.loc[mask2,[*data.columns[2:6292]]].applymap(lambda x:getter(x)))#xtrain->particpants hr values
print(data.loc[mask2,"Target"])#ytrain-> target emo


0        BASELINE
3         NEUTRAL
6             AWE
9         DISGUST
12       SURPRISE
          ...    
1330         FEAR
1333    AMUSEMENT
1336      SADNESS
1339      NEUTRAL
1342          AWE
Name: Target, Length: 462, dtype: object


In [6]:
xtrain=data.loc[mask2,[*data.columns[2:6292]]].applymap(lambda x:getter(x))
ytrain=data.loc[mask2,"Target"]

In [7]:
x_train=[]
y_train=[]
label_encoder = LabelEncoder()
x_train,y_train=np.array(xtrain),np.array(ytrain)
y_train = label_encoder.fit_transform(y_train)
label_encoder_name_mapping = dict(zip(label_encoder.classes_,
                                         label_encoder.transform(label_encoder.classes_)))
x_train,  x_test, y_train, y_test = train_test_split(x_train, y_train, train_size = 0.7, random_state = 123)
x_train,x_test=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1)),np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
#print(label_encoder_name_mapping)
print(x_train)

[[[88]
  [88]
  [88]
  ...
  [ 0]
  [ 0]
  [ 0]]

 [[76]
  [76]
  [76]
  ...
  [ 0]
  [ 0]
  [ 0]]

 [[89]
  [89]
  [88]
  ...
  [ 0]
  [ 0]
  [ 0]]

 ...

 [[95]
  [95]
  [95]
  ...
  [ 0]
  [ 0]
  [ 0]]

 [[69]
  [70]
  [69]
  ...
  [ 0]
  [ 0]
  [ 0]]

 [[72]
  [72]
  [72]
  ...
  [ 0]
  [ 0]
  [ 0]]]


In [8]:
#initalizing the RNN
regressor =tf.keras.Sequential()
#adding the first lstm layer and some dropout regularisation
regressor.add(tf.keras.layers.LSTM(units=50,return_sequences=True,input_shape=(x_train.shape[1],1)))
regressor.add(tf.keras.layers.Dropout(0.2))
#adding the 2nd lstm layer and some dropout regularisation
regressor.add(tf.keras.layers.LSTM(units=50,return_sequences=True))
regressor.add(tf.keras.layers.Dropout(0.2))

regressor.add(tf.keras.layers.Flatten())

#adding the output layer
regressor.add (tf.keras.layers.Dense(units=11,activation='softmax'))
#compiling the RNN
regressor.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')
print(regressor.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 6290, 50)          10400     
                                                                 
 dropout (Dropout)           (None, 6290, 50)          0         
                                                                 
 lstm_1 (LSTM)               (None, 6290, 50)          20200     
                                                                 
 dropout_1 (Dropout)         (None, 6290, 50)          0         
                                                                 
 flatten (Flatten)           (None, 314500)            0         
                                                                 
 dense (Dense)               (None, 11)                3459511   
                                                                 
Total params: 3,490,111
Trainable params: 3,490,111
Non-

In [9]:
#Fitting the RNN to the training set
history=regressor.fit(x_train,y_train,validation_split = 0.2,epochs=50,batch_size=36,callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss',
    patience=5,
    
    restore_best_weights=True)
])

Epoch 1/50
8/8 [==============================] - 43s 5s/step - loss: 5.7113 - accuracy: 0.2558 - val_loss: 2.7944 - val_accuracy: 0.2923
Epoch 2/50
8/8 [==============================] - 40s 5s/step - loss: 1.9250 - accuracy: 0.4612 - val_loss: 1.5426 - val_accuracy: 0.4769
Epoch 3/50
8/8 [==============================] - 39s 5s/step - loss: 1.1709 - accuracy: 0.6047 - val_loss: 1.3119 - val_accuracy: 0.5385
Epoch 4/50
8/8 [==============================] - 39s 5s/step - loss: 0.8138 - accuracy: 0.6240 - val_loss: 0.7943 - val_accuracy: 0.6923
Epoch 5/50
8/8 [==============================] - 40s 5s/step - loss: 0.7720 - accuracy: 0.6860 - val_loss: 0.8293 - val_accuracy: 0.6308
Epoch 6/50
8/8 [==============================] - 40s 5s/step - loss: 0.7117 - accuracy: 0.6783 - val_loss: 0.7688 - val_accuracy: 0.6923
Epoch 7/50
8/8 [==============================] - 39s 5s/step - loss: 0.6926 - accuracy: 0.6938 - val_loss: 0.7206 - val_accuracy: 0.7231
Epoch 8/50
8/8 [==================

In [11]:
model_acc = regressor.evaluate(x_test, y_test, verbose=0)[1]
print(f"Test Accuracy : {round(model_acc, 3)*100} %")
     

Test Accuracy : 65.5 %


cross validation part for particpants